In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
# Baca file teks yang berisi daftar URL
with open('links.txt', 'r') as file:
    urls = [line.strip() for line in file]

In [4]:
skipped_urls = []
urldoc = {}

for url in urls:
    # Kirim permintaan GET
    response = requests.get(url)

    # Parse konten dengan BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Temukan elemen dengan atribut itemprop="articleBody"
    article_body = soup.find(class_='main-content-detail')

    # Jika elemen tidak ditemukan, lewati url dan tambahkan ke list
    if article_body is None:
        skipped_urls.append(url)
        continue

    # Dapatkan teks dari setiap elemen <p> dan simpan ke dalam list
    paragraphs = [p.get_text() for p in article_body.find_all('p')]

    # Gabungkan list menjadi satu string, dengan setiap paragraf dipisahkan oleh baris baru
    text = '\n'.join(paragraphs)

    # Hapus tanda baca dan karakter non-alfabet
    text = re.sub(r'[^\w\s]', ' ', text)

    # Case folding
    text = text.lower()

    # Ekstrak judul halaman sebagai nama file (Anda mungkin perlu menyesuaikan ini tergantung pada struktur halaman web)
    title = soup.title.string

    # Buat versi singkat dari judul untuk digunakan sebagai nama file
    short_title = title[:100]  # Ambil 100 karakter pertama dari judul

    # Ganti karakter yang tidak valid dalam nama file
    filename = "".join(c for c in short_title if c.isalnum() or c.isspace()).rstrip() + ".txt"

    urldoc[filename] = url

    # Tentukan direktori tempat Anda ingin menyimpan file
    directory = "D:/kuliah/Semester 5/Penelusuran Informasi/PI Project 2.0/input_folder"

    # Gabungkan direktori dan nama file untuk mendapatkan path lengkap ke file
    file_path = os.path.join(directory, filename)

    # Simpan teks ke file .txt
    with open(file_path, 'w') as f:
        f.write(text)

# Save the dictionary to a file with each URL on a new line
with open('urldoc.json', 'w', encoding='utf-8') as f:
    json.dump(urldoc, f, ensure_ascii=False, indent=4)
